In [5]:
import pandas as pd
from datetime import timedelta
import os

# Load logs
df = pd.read_csv("auth_logs.csv", parse_dates=["timestamp"])

# Sort by IP and timestamp
df = df.sort_values(by=["source_ip", "timestamp"])

# Define brute force: 5+ failed logins within 5 minutes
brute_force_ips = []

for ip in df["source_ip"].unique():
    ip_df = df[df["source_ip"] == ip]
    failures = ip_df[ip_df["auth_result"] == "FAIL"]

    for i in range(len(failures) - 4):
        window = failures.iloc[i:i+5]
        time_diff = window.iloc[-1]["timestamp"] - window.iloc[0]["timestamp"]
        if time_diff <= timedelta(minutes=5):
            brute_force_ips.append(ip)
            break

# Deduplicate and prepare results
flagged = pd.DataFrame(set(brute_force_ips), columns=["suspicious_ip"])

# Save results to the current directory instead of trying to access parent directory
# Changed from "../output/flagged_ips.csv" to "flagged_ips.csv"
flagged.to_csv("flagged_ips.csv", index=False)
print(flagged)

  suspicious_ip
0    172.16.0.1
1  192.168.1.20
2  192.168.1.10
3    10.10.10.5
4   203.0.113.5
